# Data ingestion and cleaning
In this workbook, I walk through the data ingestion and cleaning processes for both the SAT & ACT test results for the 2018-2019 graudating year in the State of California. Most of the work involved dropping unnecessary columns and fixing formatting errors with numerical values. Since I'm focusing my research on the county level, I could also drop any data connected to specific school districts.

---
## SAT Test Results in California for the 2018-2019 graduating year

This first dataset was found on the California Department of Education's website at: http://www3.cde.ca.gov/researchfiles/satactap/sat19.txt

To start, we're going to load the 2019 SAT data from California into a DataFrame.

In [1]:
import pandas as pd
import numpy as np

sat_2019_ca = pd.read_csv('../data/sat_2019_ca.csv')
sat_2019_ca.tail()

,CDS,CCode,CDCode,SCode,RType,SName,DName,CName,Enroll12,NumTSTTakr12,...,NumERWBenchmark11,PctERWBenchmark11,NumMathBenchmark11,PctMathBenchmark11,TotNumBothBenchmark12,PctBothBenchmark12,TotNumBothBenchmark11,PctBothBenchmark11,Year,Unnamed: 25
2575,5.772678e+13,57.0,5772678.0,0.0,D,NaN,Davis Joint Unified,Yolo,717.0,307.0,...,454,95.78,428,90.30,268,87.30,421,88.82,2018-19,NaN
2576,7.000000e+12,7.0,700000.0,0.0,C,NaN,NaN,Contra Costa,14593.0,5412.0,...,4583,88.87,3933,76.27,2721,50.28,3853,74.71,2018-19,NaN
2577,1.700000e+13,17.0,1700000.0,0.0,C,NaN,NaN,Lake,667.0,167.0,...,94,74.60,50,39.68,65,38.92,47,37.30,2018-19,NaN
2578,3.800000e+13,38.0,3800000.0,0.0,C,NaN,NaN,San Francisco,4447.0,2256.0,...,1702,83.35,1566,76.69,1248,55.32,1482,72.58,2018-19,NaN
2579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From looking at the raw data, it was clear there were many extraneous columns here. Here are all the column headers:

In [2]:
sat_2019_ca.columns

Index(['CDS', 'CCode', 'CDCode', 'SCode', 'RType', 'SName', 'DName', 'CName',
       'Enroll12', 'NumTSTTakr12', 'NumERWBenchmark12', 'PctERWBenchmark12',
       'NumMathBenchmark12', 'PctMathBenchmark12', 'Enroll11', 'NumTSTTakr11',
       'NumERWBenchmark11', 'PctERWBenchmark11', 'NumMathBenchmark11',
       'PctMathBenchmark11', 'TotNumBothBenchmark12', 'PctBothBenchmark12',
       'TotNumBothBenchmark11', 'PctBothBenchmark11', 'Year', 'Unnamed: 25'],
      dtype='object')

First, I dropped all the internal codes in the first four columns, as well as the dummy `Unnamed` column at the end.

In [3]:
sat_2019_ca = sat_2019_ca.drop(['CDS', 'CCode','CDCode','SCode','Unnamed: 25'], axis=1)

Then, I dropped all data relevant to 11th grade test takers, since the ACT dataset only contains information about 12th grade test takers.

In [4]:
sat_2019_ca = sat_2019_ca.drop(['Enroll11', 'NumTSTTakr11', 'NumERWBenchmark11', 'PctERWBenchmark11', 'NumMathBenchmark11',
       'PctMathBenchmark11', 'TotNumBothBenchmark11', 'PctBothBenchmark11'], axis=1)

For the purposes of our research, we're only interested in analyzing data at the County level, so we can filter by `RType == C`. Let's see how many records that gives us: 

In [5]:
sat_2019_ca[sat_2019_ca['RType']=='C'].shape

(58, 13)

This makes sense, as California has 58 counties. Let's save that filter as a new DataFrame.

In [6]:
sat_2019_ca_counties = sat_2019_ca[sat_2019_ca['RType']=='C']
sat_2019_ca_counties.head()

,RType,SName,DName,CName,Enroll12,NumTSTTakr12,NumERWBenchmark12,PctERWBenchmark12,NumMathBenchmark12,PctMathBenchmark12,TotNumBothBenchmark12,PctBothBenchmark12,Year
1981,C,NaN,NaN,El Dorado,2226.0,604.0,547,90.56,458,75.83,442,73.18,2018-19
1982,C,NaN,NaN,Los Angeles,114618.0,48676.0,30081,61.80,19791,40.66,18555,38.12,2018-19
1983,C,NaN,NaN,Madera,2352.0,442.0,277,62.67,136,30.77,127,28.73,2018-19
1984,C,NaN,NaN,Yolo,2299.0,759.0,586,77.21,464,61.13,443,58.37,2018-19
1985,C,NaN,NaN,Kern,14229.0,3793.0,2334,61.53,1481,39.05,1364,35.96,2018-19


Looks like we'll need to reset the row indices here, so let's do that first. We can also sort by County Name here too.

In [7]:
sat_2019_ca_counties = sat_2019_ca_counties.sort_values(by='CName')
sat_2019_ca_counties = sat_2019_ca_counties.reset_index()

Since we're evaluating these numbers at the county level, we can drop the `SName` (School Name) and `DName` (District Name) columns. Finally, we can drop the `Year` column, since the ACT dataset is also for the 2018-2019 School Year.

In [8]:
sat_2019_ca_counties = sat_2019_ca_counties.drop(['index', 'RType', 'SName', 'DName','Year'], axis=1)
sat_2019_ca_counties.head()

,CName,Enroll12,NumTSTTakr12,NumERWBenchmark12,PctERWBenchmark12,NumMathBenchmark12,PctMathBenchmark12,TotNumBothBenchmark12,PctBothBenchmark12
0,Alameda,17567.0,5946.0,4566,76.79,3623,60.93,3458,58.16
1,Alpine,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador,305.0,53.0,44,83.02,35,66.04,33,62.26
3,Butte,2551.0,551.0,453,82.21,349,63.34,332,60.25
4,Calaveras,451.0,90.0,81,90.00,60,66.67,60,66.67


Now let's rename those columns to something more descriptive and readable.

In [9]:
sat_2019_ca_counties.columns = ['county', 'total_enrollment', 'SAT_test_takers',
                                'num_pass_SAT_ERW', 'pct_pass_SAT_ERW',
                                'num_pass_SAT_Math', 'pct_pass_SAT_Math',
                                'num_pass_SAT_Total', 'pct_pass_SAT_Total']

In [10]:
sat_2019_ca_counties.head(5)

,county,total_enrollment,SAT_test_takers,num_pass_SAT_ERW,pct_pass_SAT_ERW,num_pass_SAT_Math,pct_pass_SAT_Math,num_pass_SAT_Total,pct_pass_SAT_Total
0,Alameda,17567.0,5946.0,4566,76.79,3623,60.93,3458,58.16
1,Alpine,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador,305.0,53.0,44,83.02,35,66.04,33,62.26
3,Butte,2551.0,551.0,453,82.21,349,63.34,332,60.25
4,Calaveras,451.0,90.0,81,90.00,60,66.67,60,66.67


Almost done with pre-processing this DataFrame! Let's examine the remaining data types:

In [11]:
sat_2019_ca_counties.dtypes

county                 object
total_enrollment      float64
SAT_test_takers       float64
num_pass_SAT_ERW       object
pct_pass_SAT_ERW       object
num_pass_SAT_Math      object
pct_pass_SAT_Math      object
num_pass_SAT_Total     object
pct_pass_SAT_Total     object
dtype: object

It appears we have several numbers stored as strings, so we'll need to reformat those columns.

In [12]:
# Define funtion to map numbers stored as strings to float.
# If unidentified character, return NaN
def Str_to_float(obj):
    if obj == '*':
        return np.nan
    elif type(obj) == str:
        return float(obj)
    elif type(obj) == float:
        return obj
    else:
        return np.nan

In [13]:
# Map average score columns from strings to floats
for row in sat_2019_ca_counties.iloc[:, 3: 9]:
    print(row)
    sat_2019_ca_counties[row] = sat_2019_ca_counties[row].map(Str_to_float)

num_pass_SAT_ERW
pct_pass_SAT_ERW
num_pass_SAT_Math
pct_pass_SAT_Math
num_pass_SAT_Total
pct_pass_SAT_Total


In [14]:
# Define funtion to map numbers stored as floats to ints
# If unidentified character, return NaN
def Float_to_int(n):
    if type(n) == float:
        return int(n)
    if type(n) == int:
        return n
    else:
        return np.nan

In [15]:
for row in sat_2019_ca_counties.iloc[:, [1, 2]]:
    print(row)
    sat_2019_ca_counties[row] = sat_2019_ca_counties[row].map(Float_to_int)

total_enrollment
SAT_test_takers


Let's check that all this reformatting worked:

In [16]:
sat_2019_ca_counties.dtypes

county                 object
total_enrollment        int64
SAT_test_takers         int64
num_pass_SAT_ERW      float64
pct_pass_SAT_ERW      float64
num_pass_SAT_Math     float64
pct_pass_SAT_Math     float64
num_pass_SAT_Total    float64
pct_pass_SAT_Total    float64
dtype: object

It might be helpful to add a column that calculates the percent of SAT test-takers per county enrollment.

In [17]:
sat_2019_ca_counties.insert(3, 'pct_took_SAT',
                            round((sat_2019_ca_counties['SAT_test_takers'] / sat_2019_ca_counties['total_enrollment']), 2))

In [18]:
sat_2019_ca_counties.head()

,county,total_enrollment,SAT_test_takers,pct_took_SAT,num_pass_SAT_ERW,pct_pass_SAT_ERW,num_pass_SAT_Math,pct_pass_SAT_Math,num_pass_SAT_Total,pct_pass_SAT_Total
0,Alameda,17567,5946,0.34,4566.0,76.79,3623.0,60.93,3458.0,58.16
1,Alpine,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador,305,53,0.17,44.0,83.02,35.0,66.04,33.0,62.26
3,Butte,2551,551,0.22,453.0,82.21,349.0,63.34,332.0,60.25
4,Calaveras,451,90,0.20,81.0,90.00,60.0,66.67,60.0,66.67


Looks good! On to the next dataset.

---

# ACT Test Results in California for the 2018-2019 graduating year
This second dataset was also found on the California Department of Education's website at: http://www3.cde.ca.gov/researchfiles/satactap/act19.txt

Again, we'll start by loading the .csv file into a DataFrame.

In [19]:
act_2019_ca = pd.read_csv('../data/act_2019_ca.csv')
act_2019_ca.columns

Index(['CDS', 'CCode', 'CDCode', 'SCode', 'RType', 'SName', 'DName', 'CName',
       'Enroll12', 'NumTstTakr', 'AvgScrRead', 'AvgScrEng', 'AvgScrMath',
       'AvgScrSci', 'NumGE21', 'PctGE21', 'Year', 'Unnamed: 17'],
      dtype='object')

Looks like we've got a lot of the same extraneous columns, so we'll just repeat the data cleaning steps from above.

In [20]:
# Drop internal code columns
act_2019_ca = act_2019_ca.drop(['CDS', 'CCode','CDCode','SCode','Unnamed: 17'], axis=1)

# Filter by county level
act_2019_ca_counties = act_2019_ca[act_2019_ca['RType']=='C']

# Drop School District, School Name, and Year columns
act_2019_ca_counties = act_2019_ca_counties.drop(['RType', 'SName', 'DName','Year'], axis=1)

In [21]:
# Reset indices and sort by County name
act_2019_ca_counties = act_2019_ca_counties.sort_values(by='CName')
act_2019_ca_counties = act_2019_ca_counties.reset_index()
act_2019_ca_counties = act_2019_ca_counties.drop('index', axis = 1)

In [22]:
#Rename column headers
act_2019_ca_counties.columns = ['county', 'total_enrollment', 'ACT_test_takers',
                                'avg_ACT_Reading', 'avg_ACT_English', 'avg_ACT_Math', 'avg_ACT_Science',
                                'num_pass_ACT_Total', 'pct_pass_ACT_Total']

In [23]:
# Map average score columns from strings to floats
for row in act_2019_ca_counties.iloc[:, 3: 9]:
    print(row)
    act_2019_ca_counties[row] = act_2019_ca_counties[row].map(Str_to_float)

avg_ACT_Reading
avg_ACT_English
avg_ACT_Math
avg_ACT_Science
num_pass_ACT_Total
pct_pass_ACT_Total


In [24]:
# Map enrollment and test takers columns from floats to ints
for row in act_2019_ca_counties.iloc[:, [1, 2]]:
    print(row)
    act_2019_ca_counties[row] = act_2019_ca_counties[row].map(Float_to_int)

total_enrollment
ACT_test_takers


Test that the reformatting worked:

In [25]:
act_2019_ca_counties.dtypes

county                 object
total_enrollment        int64
ACT_test_takers         int64
avg_ACT_Reading       float64
avg_ACT_English       float64
avg_ACT_Math          float64
avg_ACT_Science       float64
num_pass_ACT_Total    float64
pct_pass_ACT_Total    float64
dtype: object

Finally, let's add that column `Percent Took Test` back into this DataFrame as well:

In [26]:
act_2019_ca_counties.insert(3, 'pct_took_ACT',
                            round((act_2019_ca_counties['ACT_test_takers'] / act_2019_ca_counties['total_enrollment']), 2))

In [27]:
act_2019_ca_counties.head()

,county,total_enrollment,ACT_test_takers,pct_took_ACT,avg_ACT_Reading,avg_ACT_English,avg_ACT_Math,avg_ACT_Science,num_pass_ACT_Total,pct_pass_ACT_Total
0,Alameda,17567,3630,0.21,25.0,24.0,24.0,24.0,2459.0,67.74
1,Alpine,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador,305,37,0.12,24.0,23.0,23.0,22.0,29.0,78.38
3,Butte,2551,226,0.09,23.0,22.0,23.0,23.0,146.0,64.60
4,Calaveras,451,31,0.07,26.0,24.0,24.0,24.0,27.0,87.10


We're ready to merge!


---

# Merging SAT & ACT Test Results
In this final section, we'll validate some of the numbers across both datasets, and merge all the columns into one DataFrame before writing it to a .csv

First, it appears that `total_enrollment` per county is the same across both the SAT and ACT datasets. Let's double check those numbers match up.

In [28]:
bool_array = []

# Test every number in SAT['Enrollment'] against it's corresponding county in ACT['Enrollment']
for i, n in enumerate(sat_2019_ca_counties['total_enrollment']):
    bool_array.append(n == act_2019_ca_counties['total_enrollment'][i])
    
if False in bool_array:
    print("There's an error somewhere.")
else:
    print("The data matches up perfectly!")

The data matches up perfectly!


That means we can drop `total_enrollment` from one of the DataFrames.

In [29]:
act_2019_ca_counties = act_2019_ca_counties.drop('total_enrollment', axis = 1)

Time to merge! We'll use an inner join on the `County` column to bring these two datasets together.

In [30]:
test_results_2019 = pd.merge(sat_2019_ca_counties, act_2019_ca_counties, how='inner', on='county')

In [31]:
test_results_2019.shape

(58, 18)

In [32]:
test_results_2019.describe()

,total_enrollment,SAT_test_takers,pct_took_SAT,num_pass_SAT_ERW,pct_pass_SAT_ERW,num_pass_SAT_Math,pct_pass_SAT_Math,num_pass_SAT_Total,pct_pass_SAT_Total,ACT_test_takers,pct_took_ACT,avg_ACT_Reading,avg_ACT_English,avg_ACT_Math,avg_ACT_Science,num_pass_ACT_Total,pct_pass_ACT_Total
count,58.000000,58.000000,57.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,58.000000,57.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,8442.241379,2938.120690,0.257719,2117.927273,74.402545,1482.545455,52.194727,1403.036364,49.723091,1425.310345,0.148070,22.648148,21.537037,21.814815,21.722222,841.759259,56.757407
std,17344.164490,7079.370425,0.088318,4612.155846,9.793826,3131.258301,13.101901,2949.818462,13.253115,3164.257498,0.095869,2.466052,2.589623,2.137483,2.041241,1675.299939,17.189004
min,0.000000,0.000000,0.080000,12.000000,52.700000,9.000000,20.270000,9.000000,18.920000,0.000000,0.010000,17.000000,15.000000,17.000000,18.000000,8.000000,20.000000
25%,525.500000,90.250000,0.200000,103.000000,67.080000,65.500000,41.105000,62.500000,38.220000,64.250000,0.090000,21.000000,20.000000,20.000000,20.000000,44.500000,43.560000
50%,2325.500000,579.500000,0.240000,547.000000,75.390000,382.000000,53.210000,348.000000,50.690000,365.000000,0.140000,23.000000,22.000000,22.000000,22.000000,202.000000,59.775000
75%,7477.250000,2343.750000,0.320000,2021.500000,82.425000,1438.000000,61.715000,1357.000000,59.825000,1212.250000,0.180000,25.000000,24.000000,23.750000,23.750000,671.250000,71.372500
max,114618.000000,48676.000000,0.510000,30081.000000,91.670000,19791.000000,75.830000,18555.000000,73.180000,20678.000000,0.630000,27.000000,27.000000,26.000000,26.000000,9182.000000,87.100000


Looking good! One last additional dataset to bring in.

---

# Adding Census Data on population per county

After some outside research, I also decided to use a dataset from the California Department of Public Health on the number of people living below 200% of the Federal Poverty level, per county.

In 2018, this level was approximately 12,100 for one person, 16,500 for a two-person household, and 25,100 for a four-person household. Due to California’s high cost of living, the state considers 200\% of the federal poverty rate to be a more realistic measure of financial hardship.

This data was derived from the U.S. Census Bureau American Community Survey, 2011-2015 Selected Population Tables table C17002 (overall poverty), and can be found online at:

https://www.cdph.ca.gov/Programs/OHE/Pages/HCI-Search.aspx


In [33]:
ca_poverty_data = pd.read_excel('../data/ca_hhs_poverty_rate_2011-2015.xlsx', sheet_name = 'Data')
ca_poverty_data.shape

(32005, 26)

Not surprisingly, it contains a lot of extraneous columns.

In [34]:
ca_poverty_data.columns

Index(['ind_id', 'ind_definition', 'reportyear', 'race_eth_code',
       'race_eth_name', 'geotype', 'geotypevalue', 'geoname', 'county_name',
       'county_fips', 'region_name', 'region_code', 'strata_one_code',
       'strata_one_name', 'strata_two_code', 'strata_two_name', 'numerator',
       'denominator', 'estimate', 'LL_95CI', 'UL_95CI', 'SE', 'RSE',
       'CA_decile', 'CA_RR', 'version'],
      dtype='object')

First, I filtered by county-level data ONLY, then dropped a bunch of columns.

In [35]:
# Filter by county-level data only for the years 2012 - 2016
ca_poverty_counties = ca_poverty_data[ca_poverty_data['geotype']=='CO']

In [36]:
# Drop unneccessary columns
ca_poverty_counties = ca_poverty_counties.drop(['ind_id', 'ind_definition', 'geotype', 'geotypevalue', 'geoname',
                                                'county_fips', 'region_name', 'region_code', 'strata_one_code',
                                                'strata_one_name', 'strata_two_code', 'strata_two_name', 'version'
                                               ], axis = 1)

Then, I filtered further by the total of all races and ethnicities, for the 2011-2015 census survey

In [37]:
ca_poverty_counties_2015 = ca_poverty_counties[
                     (ca_poverty_counties['race_eth_name']=='Total') & #pull total value for all race/ethnicities
                     (ca_poverty_counties['reportyear']=='2011-2015') ] #from report year 2011-2015

In [38]:
ca_poverty_counties_2015 = ca_poverty_counties_2015.drop(['reportyear',
                                                          'race_eth_code', 'race_eth_name',
                                                          'CA_decile'], axis = 1)

Next, I renamed all the columns for readability and descriptiveness according to the source's data dictionary.

In [39]:
ca_poverty_counties_2015.columns = ['county', 'below_200pct_poverty', 'total_pop', 'pct_estimate',
                                    'lower_bound_95CI', 'upper_bound_95CI', 'st_error', 'rel_error',
                                    'pct_above_below_state_est']

Finally, I re-indexed the counties in alphabetical order.

In [40]:
ca_poverty_counties_2015 = ca_poverty_counties_2015.reset_index()
ca_poverty_counties_2015 = ca_poverty_counties_2015.drop('index', axis = 1)

Luckily, the data is already formatted properly.

In [41]:
ca_poverty_counties_2015.dtypes

county                        object
below_200pct_poverty           int64
total_pop                      int64
pct_estimate                 float64
lower_bound_95CI             float64
upper_bound_95CI             float64
st_error                     float64
rel_error                    float64
pct_above_below_state_est    float64
dtype: object

Time to do one final merge!

In [42]:
# Generate DataFrame with select population statistics, per County
df = ca_poverty_counties_2015[['county', 'total_pop', 'below_200pct_poverty',
                               'pct_estimate', 'pct_above_below_state_est']]

# Merge standardized tests DataFrame with population
ca_tests_and_population_merged = pd.merge(test_results_2019, df, how='inner', on='county')

In [43]:
ca_tests_and_population_merged = ca_tests_and_population_merged[['county', 'total_pop', 'below_200pct_poverty',
                                'pct_estimate', 'pct_above_below_state_est', 'total_enrollment',
                                'SAT_test_takers', 'pct_took_SAT', 'ACT_test_takers', 'pct_took_ACT',
                                'num_pass_SAT_Total', 'pct_pass_SAT_Total', 'num_pass_ACT_Total', 'pct_pass_ACT_Total',
                                'num_pass_SAT_ERW', 'pct_pass_SAT_ERW', 'avg_ACT_Reading', 'avg_ACT_English',
                                'num_pass_SAT_Math', 'pct_pass_SAT_Math', 'avg_ACT_Math'
                                ]]

In [44]:
ca_tests_and_population_merged.head()

,county,total_pop,below_200pct_poverty,pct_estimate,pct_above_below_state_est,total_enrollment,SAT_test_takers,pct_took_SAT,ACT_test_takers,pct_took_ACT,...,pct_pass_SAT_Total,num_pass_ACT_Total,pct_pass_ACT_Total,num_pass_SAT_ERW,pct_pass_SAT_ERW,avg_ACT_Reading,avg_ACT_English,num_pass_SAT_Math,pct_pass_SAT_Math,avg_ACT_Math
0,Alameda,1557087,427064,27.427112,0.759022,17567,5946,0.34,3630,0.21,...,58.16,2459.0,67.74,4566.0,76.79,25.0,24.0,3623.0,60.93,24.0
1,Alpine,1115,442,39.641256,1.097037,0,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador,32867,9264,28.186327,0.780032,305,53,0.17,37,0.12,...,62.26,29.0,78.38,44.0,83.02,24.0,23.0,35.0,66.04,23.0
3,Butte,217509,95781,44.035419,1.218642,2551,551,0.22,226,0.09,...,60.25,146.0,64.60,453.0,82.21,23.0,22.0,349.0,63.34,23.0
4,Calaveras,44221,13243,29.947310,0.828766,451,90,0.20,31,0.07,...,66.67,27.0,87.10,81.0,90.00,26.0,24.0,60.0,66.67,24.0


---

# Custom columns
After initial exploratory analysis, it was discovered that it would be helpful to add a few more custom columns to help with grouping data.

To better understand the regional differences among California's numerous counties, I decided to load the Census categorical data on subregion, which can be found at: https://census.ca.gov/regions/_.

In [45]:
def find_region(value):
    
    superior = ['Butte', 'Colusa', 'El Dorado', 'Glenn', 'Lassen', 'Modoc', 'Nevada', 'Placer', 'Plumas',
            'Sacramento', 'Shasta', 'Sierra', 'Siskiyou', 'Sutter', 'Tehama', 'Yolo', 'Yuba']
    north_coast = ['Del Norte', 'Humboldt', 'Lake', 'Mendocino', 'Napa', 'Sonoma', 'Trinity']
    bay_area = ['Alameda', 'Contra Costa', 'Marin', 'San Francisco', 'San Mateo', 'Santa Clara', 'Solano']
    northern_valley = ['Alpine', 'Amador', 'Calaveras', 'Madera', 'Mariposa', 'Merced', 'Mono',
                       'San Joaquin', 'Stanislaus', 'Tuolumne']
    central_coast = ['Monterey', 'San Benito', 'San Luis Obispo', 'Santa Barbara', 'Santa Cruz', 'Ventura']
    southern_valley = ['Fresno', 'Inyo', 'Kern', 'Kings', 'Tulare']
    southern_california = ['Riverside', 'San Bernardino', 'Los Angeles', 'Orange', 'Imperial', 'San Diego']

    if value in superior:
        return 'superior'
    elif value in north_coast:
        return 'n_coast'
    elif value in bay_area:
        return 'bay_area'
    elif value in northern_valley:
        return 'n_valley'
    elif value in central_coast:
        return 'cen_coast'
    elif value in southern_valley:
        return 's_valley'
    elif value in southern_california:
        return 'so_cal'
    else:
        return ''

In [46]:
# Map counties in County column to respective region catgeory
ca_tests_and_population_merged.insert(1, 'region', ca_tests_and_population_merged['county'].map(find_region))

In [47]:
# Create boolean values if total population of county is over 900,000 or less than 75,000
# This results in a Top 10 and Bottom 20 subgroup of counties
ca_tests_and_population_merged.insert(2, 'isTop10', (ca_tests_and_population_merged['total_pop']>900_000)*1)
ca_tests_and_population_merged.insert(3, 'isBottom20', (ca_tests_and_population_merged['total_pop']<75_000)*1)

In [48]:
# Calculate new column 'total_test_takers' that is the sum of the total number of ACT & SAT test takers
ca_tests_and_population_merged.insert(9, 'total_test_takers', ca_tests_and_population_merged['SAT_test_takers']+ca_tests_and_population_merged['ACT_test_takers'])

In [49]:
# Create new column that divides total test takers by enrollment
ca_tests_and_population_merged.insert(10, 'pct_test_takers', ca_tests_and_population_merged['total_test_takers'] / ca_tests_and_population_merged['total_enrollment'])

Let's check those new columns.

In [50]:
ca_tests_and_population_merged.iloc[:, [0, 8, 9, 10]].tail(4)

,county,total_enrollment,total_test_takers,pct_test_takers
54,Tuolumne,518,103,0.198842
55,Ventura,10750,6086,0.566140
56,Yolo,2299,1114,0.484559
57,Yuba,977,242,0.247697


In [51]:
ca_tests_and_population_merged.columns

Index(['county', 'region', 'isTop10', 'isBottom20', 'total_pop',
       'below_200pct_poverty', 'pct_estimate', 'pct_above_below_state_est',
       'total_enrollment', 'total_test_takers', 'pct_test_takers',
       'SAT_test_takers', 'pct_took_SAT', 'ACT_test_takers', 'pct_took_ACT',
       'num_pass_SAT_Total', 'pct_pass_SAT_Total', 'num_pass_ACT_Total',
       'pct_pass_ACT_Total', 'num_pass_SAT_ERW', 'pct_pass_SAT_ERW',
       'avg_ACT_Reading', 'avg_ACT_English', 'num_pass_SAT_Math',
       'pct_pass_SAT_Math', 'avg_ACT_Math'],
      dtype='object')

Now we're ready to save our final dataset to a new .csv file!!

In [53]:
ca_tests_and_population_merged.to_csv('../data/ca_tests_and_population_merged.csv')